In [1]:
import numpy as np

def linear_layer(x, w, b):
    out = x @ w + b
    return out

def relu(x):
    """
    对输入张量 x 执行元素级的 ReLU (Rectified Linear Unit) 操作。
    公式为: f(x) = max(0, x)
    """
    # ===== 在此实现 =====
    # pass
    return np.maximum(0, x)
    

def flatten(x):
    """
    将一个四维张量 (N, C, H, W) 展平为一个二维张量 (N, C*H*W)。
    N 是批量大小，需要保持不变。
    """
    # ===== 在此实现 =====
    N, C, H, W = x.shape
    return x.reshape(N, C*H*W)

In [2]:
x = np.array([[-2], [-1], [0], [1], [2]])

w1, b1 = np.array([[2]]), np.array([-1])
w2, b2 = np.array([[-1]]), np.array([0.5])

linear1 = linear_layer(x, w1, b1)
linear2 = linear_layer(x, w2, b2)
relu1 = relu(linear1)
relu2 = relu(linear2)
print ('linear2:', linear2)
print ('relu2:', relu2)

linear2: [[ 2.5]
 [ 1.5]
 [ 0.5]
 [-0.5]
 [-1.5]]
relu2: [[2.5]
 [1.5]
 [0.5]
 [0. ]
 [0. ]]


**分析与思考：**

1. **观察与对比**：对比B中relu前和relu后的值，`relu` 函数具体做了什么？对比A和B的最终输出，它们的输出模式有何根本不同？
2. **总结**：通过本次实验，请用你自己的话讲解，为什么非线性激活函数是构建深度神经网络的**必需品**？
3. **扩展思考**：本题中我们基本没有涉及`Flatten`层，仅将其实现以为后续使用，`Flatten` 层虽然简单，但它在CNN中通常扮演着什么角色？

**回答：**
1. 观察与对比:ReLU 将所有负数部分直接截断为 0，仅保留正数部分，并保持其原值。
2. 将神经网络变为非线性网络，不然加再多神经元都是线性的。
3. 把多维特征展平为一维向量 (C*H*W)，从而可以输入到全连接层中，为分类或回归做准备。

参数灾难**：假设输入一张 `100x100` 的单通道图，一个全连接层需要多少权重才能仅仅让**一个输出神经元**连接到所有输入像素？作为对比，一个 `3x3` 的卷积核总共需要多少个权重参数？

答：（1）10001个 （2）10个

In [3]:
def conv2d(x, w, b, stride=1, padding=0):
    """
    使用循环实现一个朴素的 2D 卷积操作。
    """
    # ===== 在此实现 =====
    N, C, H, W = x.shape
    F, _, KH, KW = w.shape
    
    if padding > 0:
        x_padded = np.pad(x, ((0,0),(0,0),(padding,padding),(padding,padding)), mode='constant')
    else:
        x_padded = x
    
    H_out = (H + 2*padding - KH) // stride + 1
    W_out = (W + 2*padding - KW)//stride + 1
    
    out = np.zeros((N, F, H_out, W_out))
    
    for n in range(N):
        for f in range(F):
            for i in range(H_out):
                for j in range(W_out):
                    h_start = i * stride
                    w_start = j * stride
                    h_end = h_start + KH
                    w_end = w_start + KW
                    # 卷积：对应通道相乘再累加
                    out[n, f, i, j] = np.sum(
                        x_padded[n, :, h_start:h_end, w_start:w_end] * w[f, :, :, :]
                    ) + b[f]
    
    return out  

In [4]:
image_centered1 = np.array([[
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0]
    ]], dtype=np.float32).reshape(1, 1, 5, 5)

w = np.array([
    [0,1,0],
    [1,1,1],
    [0,1,0]
], dtype=np.float32).reshape(1,1,3,3)

b = [0.0]

In [5]:
conv1 = conv2d(image_centered1, w, b)
conv1

array([[[[2., 2., 2.],
         [2., 5., 2.],
         [2., 2., 2.]]]])

In [6]:
image_centered2 = np.array([[
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1],
        [0, 0, 0, 1, 0]
    ]], dtype=np.float32).reshape(1, 1, 5, 5)
conv2 = conv2d( image_centered2, w, b)
conv2

array([[[[0., 0., 1.],
         [0., 2., 2.],
         [1., 2., 5.]]]])

**局部性：** 卷积核每次只能看见它大小的这块区域，而不是一次看到整张图

**平移不变性：** 因为是卷积核在图片上横向滑动，并且每次滑动都与前面的参数是共享的，图片上的特征就算平移了也只会改变提取到后面感受野的位置。

In [8]:
def max_pool2d(x, kernel_size=2, stride=2):
    # ===== 在此实现 =====
    N, F, H, W = x.shape
    
    H_out = (H  - kernel_size) // stride + 1
    W_out = (W - kernel_size)//stride + 1
    
    out = np.zeros((N, F, H_out, W_out))

    for n in range(N):
        for f in range(F):
            for i in range(H_out):
                for j in range(W_out):
                    h_start = i * stride
                    w_start = j * stride
                    h_end = h_start + kernel_size
                    w_end = w_start + kernel_size
                    window = x[n, f, h_start:h_end, w_start:w_end]
                    out[n,f,i,j] = np.max(window)
    return out

def mean_pool2d(x, kernel_size=2, stride=2):
    # ===== 在此实现 =====
    N, F, H, W = x.shape
    
    H_out = (H  - kernel_size) // stride + 1
    W_out = (W - kernel_size)//stride + 1
    
    out = np.zeros((N, F, H_out, W_out))

    for n in range(N):
        for f in range(F):
            for i in range(H_out):
                for j in range(W_out):
                    h_start = i * stride
                    w_start = j * stride
                    h_end = h_start + kernel_size
                    w_end = w_start + kernel_size
                    window = x[n, f, h_start:h_end, w_start:w_end]
                    out[n,f,i,j] = np.mean(window)
    return out

pool1 = max_pool2d(conv1)
pool2 = max_pool2d(conv2)
mean_pool1 = mean_pool2d(conv1)
mean_pool2 = mean_pool2d(conv2)
print ('max_pool1', pool1)
print ('max_pool2', pool2)
print ('mean_pool1',mean_pool1)
print ('mean_pool2',mean_pool2)

max_pool1 [[[[5.]]]]
max_pool2 [[[[2.]]]]
mean_pool1 [[[[2.75]]]]
mean_pool2 [[[[0.5]]]]


**分析与思考：**

1. **稳健性**：对比两个特征图的输出，这个实验如何证明了最大池化层能提供一定程度的“平移不变性”？
2. **降维与效率**：对比池化前后的特征图尺寸，你认为池化层对整个网络的计算效率有什么好处？
3. **机制对比**：请思考，如果将最大池化其换成“平均池化”（Average Pooling），实验结果会有何不同？在筛选特征方面，最大池化和平均池化各自的倾向是什么？

**回答：**
1.最大池化取局部窗口内的最大值，只要特征仍在池化窗口内移动，输出值通常不变。但有些特殊情况，比如明显的特征在边缘没池化到，或者被更明显的特征掩盖了，可能会影响平移不变性。

2.后续卷积层输入更小，但总体前向传播/反向传播计算量下降。

3.**最大池化:** 保留局部最显著的特征（边缘、角点）,常应用在分类/检测时，强调显著特征。

**平均池化：** 平滑局部特征，强调平均趋势，但可能丢失局部高响应信息，更适合降噪、平滑特征或生成任务。

In [41]:
import numpy as np
import gzip
import os

# (在此之前应有已实现的 conv2d, relu, max_pool2d, flatten,linear_layer函数)
# 固定随机种子，保证权重初始化一致
np.random.seed(114514)

def softmax(logits):
    """
    实现Softmax函数
    """
    # ===== 在此实现 =====
    logits = np.array(logits)
    logits = logits - np.max(logits, axis=1, keepdims=True)
    exp_scores = np.exp(logits)
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    

# --- MNIST 数据集读取函数 ---
def read_images(filename):
    """
    读取MNIST图像文件
    参数:
      filename: MNIST图像文件路径
    返回:
      images: 图像数组列表
    """
    with open(filename, 'rb') as file:
        magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
        if magic != 2051:
            raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
        
        image_data = array("B", file.read())
        
    images = []
    for i in range(size):
        img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
        img = img.reshape(rows, cols)
        images.append(img)
    
    return images


In [42]:
class TinyCNN_for_MNIST:
# ===== 在此实现你的类 =====
    def  __init__(self):
        self.conv_W = np.random.randn(4, 1, 3, 3) * 0.01
        self.conv_b = np.zeros(4)
        
        # Linear: 4*14*14 = 784, 输出 10 类
        self.fc_W = np.random.randn(4*14*14, 10) * 0.01
        self.fc_b = np.zeros(10) 
        
    def forward(self, x):
        
        out = conv2d(x, self.conv_W, self.conv_b, stride=1, padding=1)
        out = relu(out)
        out = max_pool2d(out, kernel_size=2, stride=2)
        out = flatten(out)
        logits = linear_layer(out, self.fc_W, self.fc_b)
        probs = softmax(logits)
        
        return logits, probs

In [46]:
import struct
from array import array

# --- 测试脚本 ---
if __name__ == "__main__":
    # 1. 设置 MNIST 测试集文件路径
    # !! 请将此路径修改为你自己的文件路径
    mnist_test_file = r'E:\0workspace\AI派2025招新第一轮测试\mnist\t10k-images.idx3-ubyte'

    if not os.path.exists(mnist_test_file):
        print(f"错误：找不到 MNIST 测试集文件 '{mnist_test_file}'")
    else:
        # 2. 加载所有测试图像
        test_images = read_images(mnist_test_file)
        # 3. 选取第一张图像作为测试输入
        first_test_image = test_images[0]
        # 4. 预处理图像
        input_tensor = (first_test_image.astype(np.float32) / 255.0 - 0.5) * 2.0
        input_tensor = np.expand_dims(input_tensor, axis=(0, 1))
        # 5. 实例化模型并执行前向传播
        model = TinyCNN_for_MNIST()
        logits, probs = model.forward(input_tensor)

        print("Input Tensor Shape:", input_tensor.shape)
        print("Logits shape:", logits.shape, "Probs shape:", probs.shape)
        np.set_printoptions(precision=8, suppress=False)
        print("\nLogits:", logits[0])
        print("Probs:", probs[0])
        print("\nChecksum logits sum:", float(np.sum(logits)))
        print("Checksum probs sum:", float(np.sum(probs)))

Input Tensor Shape: (1, 1, 28, 28)
Logits shape: (1, 10) Probs shape: (1, 10)

Logits: [ 0.00353601 -0.00597512 -0.01245221  0.00474773 -0.00172211  0.00710951
  0.00555682 -0.00088172  0.00028607 -0.00289982]
Probs: [0.10037968 0.09942948 0.09878755 0.10050138 0.09985325 0.10073903
 0.10058273 0.0999372  0.10005398 0.09973572]

Checksum logits sum: -0.0026948397355464285
Checksum probs sum: 0.9999999999999998
